In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
    
import numpy as np
import os
import sys
from pathlib import Path

sys.path.append('../')
from meta_fusion.benchmarks import *
from meta_fusion.methods import *
from meta_fusion.models import *
from meta_fusion.utils import *
from meta_fusion.third_party import *
from meta_fusion.config import load_config
from meta_fusion.real_data import PrepareNeuronData

In [2]:
data_name = "Neuron"
rat_name = "Mitt"
output_dim = 4
seed = 1

# Model parameters
num_modalities = 2
if rat_name == "Stella":
    mod_outs = [[300, 250, 200, 0],[20, 15, 10, 5, 0]]
elif rat_name ==  "Superchris":
    mod_outs = [[100, 0],[40, 35, 30, 25, 20, 0]]
elif rat_name ==  "Mitt":
    mod_outs = [[100, 0],[70, 65, 60, 55, 50, 40, 0]]
elif rat_name ==  "Barat":
    mod_outs = [[300, 250, 200, 0],[40, 30, 20, 15, 0]]
elif rat_name ==  "Buchanan":
    mod_outs = [[250, 200, 150, 0],[38, 35, 30, 25, 0]]
combined_hiddens = [64, 32]
mod_hiddens = [[], []]

extractor_type = 'PCA'
if extractor_type == 'encoder':
    separate=False
    is_mod_static=[False]*num_modalities
    freeze_mod_extractors=[False]*num_modalities
elif extractor_type == "separate":
    separate=True
    is_mod_static=[False]*num_modalities
    freeze_mod_extractors=[False]*num_modalities
elif extractor_type == 'PCA':
    separate=False
    is_mod_static=[True]*num_modalities
    freeze_mod_extractors=[False]*num_modalities

# Load default model configurations 
config = load_config('../experiments_real/config.json')
extractor_config = load_config('../experiments_real/config_extractor.json')

# Model files directory
ckpt_dir = f"./checkpoints/{data_name}/{rat_name}/{extractor_type}_seed{seed}/"
config['ckpt_dir'] = extractor_config['ckpt_dir'] = ckpt_dir

# Update other training parameters
config['output_dim'] = extractor_config['output_dim'] = output_dim
config["init_lr"] = 0.001
config["epochs"] = 100
config["rho_list"] = [0, 0.99]
#config["epochs"] = 2
config["ensemble_methods"] = [
        "simple_average",
        "weighted_average",
        "majority_voting",
        "weighted_voting",
#        "meta_learner",
        "greedy_ensemble"
        ]
extractor_config["init_lr"] = [0.0001, 0.001]
extractor_config["weight_decay"] = [0,0]
extractor_config["epochs"] = 50
extractor_config["verbose"] = True

In [3]:
random_state = seed
data_preparer = PrepareNeuronData(rat_name,test_size = 0.1, val_size = 0.25, task_type="multiclass", classes_to_remove=[4], oversample=False, is_graph=False,reduce_dim=True, n_lfp_features=15)
train_loader, val_loader, test_loader = data_preparer.get_dataloaders(random_state=random_state)

Original multiclass class distribution:
  Class 0: 66 trials
  Class 1: 57 trials
  Class 2: 47 trials
  Class 3: 37 trials
  Class 4: 23 trials
Performing dimension reduction...
  LFP data reduced to shape: (230, 21, 15)
  Spike data reduced to average firing rates
  Kept 72 active neurons out of 104 total neurons
  Spike data reduced to shape: (230, 72, 1)
Removing classes: [4]
Filtered 23 trials
Remaining trials: 207
Class distribution after filtering:
  Class 0: 66 trials
  Class 1: 57 trials
  Class 2: 47 trials
  Class 3: 37 trials

Class distribution:
  Train: {0: 43, 1: 37, 2: 30, 3: 24}
  Validation: {0: 17, 1: 14, 2: 12, 3: 9}
  Test: {0: 6, 1: 6, 2: 5, 3: 4}


In [4]:
data_info = data_preparer.get_data_info()
for key, value in data_info.items():
    print(f"{key}: {value}")
dim_modalities = data_info['dim_modalities']

rat_name: Mitt
dim_modalities: [315, 72]
is_graph: False
n_trials: 207
train_num: 134
val_num: 52
test_num: 21
n_nodes_lfp: 21
dim_lfp: 15
n_nodes_spk: 72
dim_spk: 1
n_classes: 4
class_distribution: {0: 66, 1: 57, 2: 47, 3: 37}
task_type: multiclass
oversample: False
time_range: (200, 250)
adjacency_threshold: [0.4, 0.2]


In [5]:
# set_random_seed(random_state)

# meta_extractor = GraphExtractors(mod_layers, input_dim, output_dim, train_loader, val_loader, 
#                                  latent_dim = latent_dim, hidden_dim=hidden_dim, pooling='topK', model_type="GCN", dropout=[0,0])
# _ = meta_extractor.get_extractors(config=extractor_config)
set_random_seed(0)

# extractor_type='PCA'
# separate=False
# is_mod_static=[True]*num_modalities
# freeze_mod_extractors=[False]*num_modalities

meta_extractor = Extractors(mod_outs, dim_modalities, train_loader, val_loader)

In [6]:
#_ = meta_extractor.get_PCA_extractors()
_ = meta_extractor.get_encoder_extractors(mod_hiddens, separate=separate, config=extractor_config)


Epoch: 1/50 - LR: [0.0001, 0.001]


100%|███████████████████████████████████████████████████████████████████████████████| 134/134 [00:00<00:00, 199.89it/s]


mod1_encoder: train loss: 1.379 - val loss: 1.378 [*] Best so far
mod2_encoder: train loss: 1.392 - val loss: 1.385 [*] Best so far

Epoch: 2/50 - LR: [0.0001, 0.001]


100%|███████████████████████████████████████████████████████████████████████████████| 134/134 [00:00<00:00, 200.66it/s]


mod1_encoder: train loss: 1.373 - val loss: 1.377 [*] Best so far
mod2_encoder: train loss: 1.383 - val loss: 1.378 [*] Best so far

Epoch: 3/50 - LR: [0.0001, 0.001]


...

mod1_encoder: train loss: 1.133 - val loss: 1.358
mod2_encoder: train loss: 0.947 - val loss: 1.150

Epoch: 49/50 - LR: [0.0001, 0.001]


100%|███████████████████████████████████████████████████████████████████████████████| 134/134 [00:00<00:00, 239.86it/s]


mod1_encoder: train loss: 1.127 - val loss: 1.358 [*] Best so far
mod2_encoder: train loss: 0.882 - val loss: 1.151

Epoch: 50/50 - LR: [0.0001, 0.001]


100%|███████████████████████████████████████████████████████████████████████████████| 134/134 [00:00<00:00, 188.85it/s]


mod1_encoder: train loss: 1.121 - val loss: 1.358
mod2_encoder: train loss: 0.883 - val loss: 1.156


In [7]:
meta_cohort = Cohorts(extractors=meta_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim,
                          is_mod_static=is_mod_static, freeze_mod_extractors=freeze_mod_extractors)

In [8]:
cohort_models = meta_cohort.get_cohort_models()
_, dim_pairs = meta_cohort.get_cohort_info()
dim_pairs

[(100, 70),
 (100, 65),
 (100, 60),
 (100, 55),
 (100, 50),
 (100, 40),
 (100, 0),
 (0, 70),
 (0, 65),
 (0, 60),
 (0, 55),
 (0, 50),
 (0, 40)]

In [9]:
cohort_models

[Fuse_Net(
   (combined_model): MLP_Net(
     (model): Sequential(
       (0): Linear(in_features=170, out_features=64, bias=True)
       (1): ReLU()
       (2): Linear(in_features=64, out_features=32, bias=True)
       (3): ReLU()
       (4): Linear(in_features=32, out_features=4, bias=True)
     )
   )
 ),
 Fuse_Net(
   (combined_model): MLP_Net(
     (model): Sequential(
       (0): Linear(in_features=165, out_features=64, bias=True)
       (1): ReLU()
       (2): Linear(in_features=64, out_features=32, bias=True)
       (3): ReLU()
       (4): Linear(in_features=32, out_features=4, bias=True)
     )
   )
 ),
 Fuse_Net(
   (combined_model): MLP_Net(
     (model): Sequential(
       (0): Linear(in_features=160, out_features=64, bias=True)
       (1): ReLU()
       (2): Linear(in_features=64, out_features=32, bias=True)
       (3): ReLU()
       (4): Linear(in_features=32, out_features=4, bias=True)
     )
   )
 ),
 Fuse_Net(
   (combined_model): MLP_Net(
     (model): Sequential(
   

In [62]:
metafuse = Trainer(config, cohort_models, [train_loader, val_loader])
metafuse.train()

Start training student cohort...
Training with disagreement penalty = 0

Epoch: 1/100 - LR: 0.001000


100%|████████████████████████████████████████████████████████████████████████████████| 134/134 [00:02<00:00, 60.93it/s]


model_1: train loss: 1.376, train task loss: 1.376 - val loss: 1.368, val task loss: 1.368 [*] Best so far
Created directory: ./checkpoints/Neuron/Mitt/PCA_seed1/0
model_2: train loss: 1.382, train task loss: 1.382 - val loss: 1.383, val task loss: 1.383 [*] Best so far
model_3: train loss: 1.422, train task loss: 1.422 - val loss: 1.419, val task loss: 1.419 [*] Best so far
model_4: train loss: 1.385, train task loss: 1.385 - val loss: 1.385, val task loss: 1.385 [*] Best so far
model_5: train loss: 1.384, train task loss: 1.384 - val loss: 1.378, val task loss: 1.378 [*] Best so far
model_6: train loss: 1.388, train task loss: 1.388 - val loss: 1.392, val task loss: 1.392 [*] Best so far
model_7: train loss: 1.382, train task loss: 1.382 - val loss: 1.377, val task loss: 1.377 [*] Best so far
model_8: train loss: 1.394, train task loss: 1.394 - val loss: 1.393, val task loss: 1.393 [*] Best so far
model_9: train loss: 1.382, train task loss: 1.382 - val loss: 1.381, val task loss: 1.

100%|████████████████████████████████████████████████████████████████████████████████| 134/134 [00:02<00:00, 59.42it/s]


model_1: train loss: 1.363, train task loss: 1.363 - val loss: 1.365, val task loss: 1.365 [*] Best so far
model_2: train loss: 1.368, train task loss: 1.368 - val loss: 1.380, val task loss: 1.380 [*] Best so far
model_3: train loss: 1.406, train task loss: 1.406 - val loss: 1.413, val task loss: 1.413 [*] Best so far
model_4: train loss: 1.372, train task loss: 1.372 - val loss: 1.381, val task loss: 1.381 [*] Best so far
model_5: train loss: 1.367, train task loss: 1.367 - val loss: 1.373, val task loss: 1.373 [*] Best so far
model_6: train loss: 1.374, train task loss: 1.374 - val loss: 1.389, val task loss: 1.389 [*] Best so far
model_7: train loss: 1.366, train task loss: 1.366 - val loss: 1.373, val task loss: 1.373 [*] Best so far
model_8: train loss: 1.390, train task loss: 1.390 - val loss: 1.389, val task loss: 1.389 [*] Best so far
model_9: train loss: 1.379, train task loss: 1.379 - val loss: 1.378, val task loss: 1.378 [*] Best so far
model_10: train loss: 1.382, train ta

...

model_1: train loss: 0.339, train task loss: 0.220 - val loss: 2.747, val task loss: 1.812
model_2: train loss: 0.337, train task loss: 0.220 - val loss: 2.779, val task loss: 1.800
model_3: train loss: 0.337, train task loss: 0.218 - val loss: 2.675, val task loss: 1.791
model_4: train loss: 0.337, train task loss: 0.219 - val loss: 2.660, val task loss: 1.730
model_5: train loss: 0.337, train task loss: 0.219 - val loss: 2.733, val task loss: 1.798
model_6: train loss: 0.336, train task loss: 0.219 - val loss: 2.864, val task loss: 1.881
model_7: train loss: 0.338, train task loss: 0.221 - val loss: 2.870, val task loss: 1.860
model_8: train loss: 0.556, train task loss: 0.548 - val loss: 1.030, val task loss: 1.019
model_9: train loss: 0.548, train task loss: 0.540 - val loss: 1.005, val task loss: 0.996
model_10: train loss: 0.525, train task loss: 0.512 - val loss: 0.986, val task loss: 0.970
model_11: train loss: 0.570, train task loss: 0.556 - val loss: 1.019, val task loss: 0.9

100%|████████████████████████████████████████████████████████████████████████████████| 134/134 [00:13<00:00, 10.18it/s]


model_1: train loss: 0.339, train task loss: 0.214 - val loss: 2.740, val task loss: 1.802
model_2: train loss: 0.336, train task loss: 0.214 - val loss: 2.752, val task loss: 1.776
model_3: train loss: 0.338, train task loss: 0.213 - val loss: 2.648, val task loss: 1.765
model_4: train loss: 0.336, train task loss: 0.213 - val loss: 2.635, val task loss: 1.702
model_5: train loss: 0.338, train task loss: 0.212 - val loss: 2.676, val task loss: 1.752
model_6: train loss: 0.337, train task loss: 0.214 - val loss: 2.836, val task loss: 1.853
model_7: train loss: 0.337, train task loss: 0.214 - val loss: 2.876, val task loss: 1.850
model_8: train loss: 0.555, train task loss: 0.547 - val loss: 1.029, val task loss: 1.019
model_9: train loss: 0.548, train task loss: 0.540 - val loss: 1.005, val task loss: 0.995
model_10: train loss: 0.523, train task loss: 0.511 - val loss: 0.982, val task loss: 0.966
model_11: train loss: 0.570, train task loss: 0.557 - val loss: 1.020, val task loss: 1.0

100%|████████████████████████████████████████████████████████████████████████████████| 134/134 [00:12<00:00, 10.38it/s]


model_1: train loss: 0.354, train task loss: 0.217 - val loss: 2.733, val task loss: 1.811
model_2: train loss: 0.353, train task loss: 0.216 - val loss: 2.753, val task loss: 1.800
model_3: train loss: 0.352, train task loss: 0.216 - val loss: 2.651, val task loss: 1.788
model_4: train loss: 0.353, train task loss: 0.216 - val loss: 2.633, val task loss: 1.729
model_5: train loss: 0.353, train task loss: 0.217 - val loss: 2.674, val task loss: 1.777
model_6: train loss: 0.352, train task loss: 0.214 - val loss: 2.828, val task loss: 1.875
model_7: train loss: 0.355, train task loss: 0.213 - val loss: 2.864, val task loss: 1.867
model_8: train loss: 0.559, train task loss: 0.551 - val loss: 1.038, val task loss: 1.027
model_9: train loss: 0.552, train task loss: 0.544 - val loss: 1.015, val task loss: 1.005
model_10: train loss: 0.527, train task loss: 0.515 - val loss: 0.996, val task loss: 0.980
model_11: train loss: 0.574, train task loss: 0.561 - val loss: 1.027, val task loss: 1.0

In [63]:
res = metafuse.test(test_loader)

Method: (simple_average), Test_Accuracy: 0.5714285714285714
Method: (weighted_average), Test_Accuracy: 0.5714285714285714
Method: (majority_voting), Test_Accuracy: 0.6190476190476191
Method: (weighted_voting), Test_Accuracy: 0.6666666666666666
Method: (greedy_ensemble), Test_Accuracy: 0.5714285714285714
Method: (best_single), Test_Accuracy: 0.5714285714285714
Method: (cohort), Test_Accuracy: [0.47619047619047616, 0.5238095238095238, 0.42857142857142855, 0.47619047619047616, 0.42857142857142855, 0.47619047619047616, 0.38095238095238093, 0.6190476190476191, 0.6190476190476191, 0.5714285714285714, 0.6666666666666666, 0.5714285714285714, 0.47619047619047616]


In [112]:
metafuse.model_num

5